## Knapsack zero-one quiz generator


In [11]:
%%html
<style> table {display: inline-block;}
</style>

Generate non trivial variants for [knapsack](https://en.wikipedia.org/wiki/Knapsack_problem) assignment.

### Assignment example:
You have some ideas on which research topics to include in your school project. You certainly will not have time to implement all the ideas. You consulted with a teacher, and for each idea, he assessed the complexity of its implementation, as well as its academic value on a five-point scale (shown in the table).

In total, you are not ready to spend more than 1 hour per day on the project, so in two weeks you have only 14 hours. What ideas are worth the time to maximize the total academic value of the project?


|Idea | Hours | Value | 
|:-- | -- | -- | 
| A | 5 | 3 | 
| B | 10 | 5 | 
| C | 6 | 4 | 
| D | 5 | 3 | 

Encoded:

[ 5, 10, 6, 5] 	 (3, 5, 4, 3) 	

Solution:

#1 #3 (A and C)


In [12]:
from itertools import chain, combinations

def brute_gen(n):
    """ Generate all combinations.
    # (0, 1), (0, 2), (1, 2), (0, 1, 2),... 
    """
    ss = list(range(0,n))
    return chain(*map(lambda x: combinations(ss, x), range(1, len(ss)+1)))

[x for x in brute_gen(3)]

[(0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]

In [13]:
# problem example
# w - weights (costs)
# v - values (profits)
# W - weight limit

w = [5, 10, 6, 5] 
v = [3, 5, 4, 3] 
W = 14
n = len(v) 

In [14]:
from collections import defaultdict

def knapsack_brute(w,v,W):
    """ Solve knapsack problem in brute force way. 
        Finds all solutions
    """
    choices = []
    maxval = 0
    for idxs in brute_gen(n):
        summ = sum(map(w.__getitem__, idxs))
        if summ <= W:
            val = sum(map(v.__getitem__, idxs))
            if val < maxval:
                continue
            elif val > maxval:
                maxval = val
                choices = [idxs,]
            elif val == maxval:
                choices.append(idxs)
    
    return maxval, choices

knapsack_brute(w,v,W)

(7, [(0, 2), (2, 3)])

In [15]:
from itertools import combinations_with_replacement

def gen_uniq(vmin=5, vmax=10, length=4):
    """ Generate all tuple variants with specified limits. """
    r = list(range(vmin, vmax+1))
    return combinations_with_replacement(r, length)

#[x for x in gen_uniq(5,10,4)]

In [16]:
import numpy as np

def boring_solution(vals, weights, idx):
    """ Check if solution is trivial 
        (items with most valuable/weight ratio are the best option).
    """
    coeff = [v/w for v,w in zip(vals,weights)]
    n = len(idx)
    max_coeff_idx = np.argpartition(coeff, -n)[-n:]
    
    return set(max_coeff_idx) == set(idx)

In [17]:
boring_solution([5,5,6,10], [2,3,4,5], [1,2])

False

In [18]:
W = 14  # Wetigh limit
N = 5   # Number of elements
N_SOL = 3 # Number of elements in solution

from operator import itemgetter

# Generate problem variants with not trivial solution

def vals_for_weights(weights):
    for vals in gen_uniq(2, 5, N):
        #print(weights, vals)
        _, solutions = knapsack_brute(weights, vals, W)
        
        if len(solutions) != 1: # multiple or zero solutions
            continue
            
        idx = solutions[0]
        if len(idx) != N_SOL: # not desired number of elements in solution
            continue
            
        if boring_solution(vals, weights, idx): # solution is trivial
            continue

        if len(set(vals)) < 3: # too fiew variants of values  
            continue
            
        if 5 not in vals:  # * maximal value not used
            continue
        
        idx_vals = itemgetter(*idx)(vals)
        if idx_vals.count(idx_vals[0]) == len(idx_vals): # all values are the same
            continue
            
        maxweight_idxs = [i for i, j in enumerate(weights) if j == max(weights)]
        # * check if the maximum weight in solution (intuitive)
        #if set(maxweight_idxs).intersection(idx): 
        #    continue
        
        maxval_idxs = [i for i, j in enumerate(vals) if j == max(vals)]
        # * check if the maximum value in solution (intuitive)
        if set(maxval_idxs).intersection(idx): 
            continue
       
        #print()
        print([x * 1 for x in weights],'\t',vals,'\t',
                      " ".join(["#{}".format(_ + 1) for _ in idx]))
        
        
for weights in gen_uniq(3, 10, N):
    if len(set(weights)) < 3: # a fiew variants of weights  
        continue
        
    #if max(weights) + min(weights) > W: # not intrigueing =)
        #continue
    if weights.count(max(weights)) > 1: # max weight occurs several times
        continue
        
    vals_for_weights(weights)

# [ weights ] 	 ( values ) 	 solution

[3, 3, 3, 6, 7] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 7] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 8] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 8] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 9] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 9] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 10] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 6, 10] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 8] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 8] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 9] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 9] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 10] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 7, 10] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 8, 9] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 8, 9] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 3, 8, 10] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 3, 8, 10] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 4, 5, 6] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 4, 5, 6] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 3, 4, 6, 7] 	 (2, 3, 3, 4, 5) 	 #2 #3 #4
[3, 3, 4, 6, 7] 	 (2, 3, 4, 4, 5) 	 #2 #3 #4
[3, 

[3, 4, 7, 8, 9] 	 (3, 3, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 9] 	 (3, 4, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 2, 3, 3, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 2, 3, 4, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 2, 4, 4, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 2, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 3, 3, 3, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 3, 3, 4, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 3, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (2, 4, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (3, 3, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 8, 10] 	 (3, 4, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 2, 3, 3, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 2, 3, 4, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 2, 4, 4, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 2, 4, 5, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 3, 3, 3, 5) 	 #1 #2 #3
[3, 4, 7, 9, 10] 	 (2, 3, 3, 4, 5) 	 #1 #2 #3
[3, 5, 5, 5, 6] 	 (2, 2, 3, 3, 5) 	 #1 #3 #4
[3, 5, 5, 5, 6] 	 (2, 2, 3, 4, 5) 	 #1 #3 #4
[3, 5, 5, 5, 6] 	 (2, 2, 4, 4, 5) 	 #1 #3 #4
[3, 5, 5, 5, 6] 	 (2, 3, 4, 4, 5) 	 #1 

[4, 5, 5, 6, 9] 	 (2, 2, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (2, 3, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (2, 3, 3, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (2, 3, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (2, 4, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (2, 4, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (3, 3, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (3, 3, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 6, 9] 	 (3, 4, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 2, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 3, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 3, 3, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 3, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 4, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (2, 4, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (3, 3, 4, 4, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (3, 3, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 6, 10] 	 (3, 4, 4, 5, 5) 	 #1 #2 #3
[4, 5, 5, 7, 8] 	 (2, 2, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 7, 8] 	 (2, 3, 3, 3, 5) 	 #1 #2 #3
[4, 5, 5, 7, 8] 	 (2, 3, 3, 4, 5) 	 #1 #2 #3
[4, 5, 5, 7, 8] 	 (2, 3, 4, 4, 5) 	 #1 #2 #3
[

  \- Fin -